In [1]:
from ctgan.synthesizers.ae_gan import CTGANV2, AutoEncoderType
from aegan_utils import load_news, get_news_Xy, train_model
from sklearn.model_selection import train_test_split
import datetime
import pandas as pd
import numpy as np
import pickle

## news dataset

In [2]:
news_train_df, news_valid_df, news_test_df, news_discrete_columns = load_news(path="../dataset/news/")

### Vanilla AE

### Result section (R2 score on real news validation dataset)

- AE dimension
    - (256, 128)
        - -0.09
    - (256, 128, 64)
        - -0.09
    - (256, 128, 64, 32)
        - -0.04
    - Best: (256, 128, 64, 32)

- G hidden dimension
    - 64
        - -0.24
    - 128
        - -0.07
    - 256
        - -1.36
    - Best: 128

- G and D layers
  - 256 * 2
    - -0.01
  - 256 * 4
    - -0.1
  - Best: 256 * 2

- D steps (1, 5) and PAC value (4, 8, 16)
  - 1, 4
  - 1, 8
  - 1, 16
  - 5, 4
  - 5, 8
  - 5, 16

In [3]:
synthesizer = "ae_gan_vanilla"

In [4]:
model = CTGANV2(
    ae_type=AutoEncoderType.VANILLA,
    ae_dim=(256, 128),
)

In [5]:
# news_trans = model.transform(news_train_df, discrete_columns=news_discrete_columns)

In [6]:
# Save data transformer and transformed data to a file
# with open("../dataset/news/data_transformer.pkl", "wb") as fp:
#     pickle.dump(model._transformer, fp)
# with open("../dataset/news/transformed_data.pkl", "wb") as fp:
#     pickle.dump(news_trans, fp)

In [7]:
# Load data transformer and transformed data
# with open("../dataset/news/data_transformer.pkl", "rb") as fp:
#     dt = pickle.load(fp)
# with open("../dataset/news/transformed_data.pkl", "rb") as fp:
#     news_trans = pickle.load(fp)

In [8]:
# model.fit(news_trans, discrete_columns=news_discrete_columns, dt=dt, is_pre_transformed=True, epochs=1)

Training AE


100%|██████████| 100/100 [00:05<00:00, 19.82it/s]


Training AE-GAN


100%|██████████| 1/1 [00:02<00:00,  2.03s/it]


In [9]:
# now = datetime.datetime.now()
# current_time = now.strftime("%d-%m-%Y-%H-%M-%S")
# model.save(
#     f"../models/news_{synthesizer}.pkl"
# )

In [10]:
model = CTGANV2.load("../models/news_ae_gan_vanilla_2.pkl")

In [17]:
# Sample fake train and validation data
news_fake_df = model.sample(news_train_df.shape[0] + news_valid_df.shape[0])
news_fake_X, news_fake_y = get_news_Xy(news_fake_df)
news_fake_train_X, news_fake_valid_X, news_fake_train_y, news_fake_valid_y = train_test_split(
    news_fake_X, news_fake_y, test_size=news_valid_df.shape[0], random_state=1, shuffle=True
)

In [6]:
# Get real validation data
news_valid_X, news_valid_y = get_news_Xy(news_valid_df)

In [22]:
# Check ML efficacy
test_score = train_model(
    news_fake_train_X,
    news_fake_train_y,
    news_fake_valid_X,
    news_fake_valid_y,
    news_valid_X,
    news_valid_y,
    input_dim=59,
    output_dim=1,
    batch_size=256,
    num_epochs=100,
    model_type="regression",
    show_print_training_score=False,
    scorer_type="r2",
    verbose=False
)
print(f"Test score: {test_score}")

  4%|▍         | 4/100 [00:00<00:05, 16.20it/s]

Epoch: 1, Training Loss: 439623277.8788, Valid Loss: 17821210.0000, Valid score: -7124.8910


 28%|██▊       | 28/100 [00:01<00:04, 16.57it/s]

Epoch: 26, Training Loss: 663.6340, Valid Loss: 855.5732, Valid score: 0.6579


 54%|█████▍    | 54/100 [00:03<00:02, 15.64it/s]

Epoch: 51, Training Loss: 596.8318, Valid Loss: 732.1336, Valid score: 0.7073


 78%|███████▊  | 78/100 [00:04<00:01, 17.29it/s]

Epoch: 76, Training Loss: 109397.5369, Valid Loss: 10846.7617, Valid score: -3.3371


100%|██████████| 100/100 [00:06<00:00, 16.36it/s]

Best validation score: 0.8390796802015947
Test score: -0.17714503125909142
